#**Limpeza dos dados**

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/projeto_final/netflix_data/ViewingActivity.csv')
df.head()

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country
0,Dimitri,2021-07-11 16:39:18,00:22:55,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #OsSuperDoidos (Episód...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:15,00:22:15,BR (Brazil)
1,Dimitri,2021-07-11 16:17:23,00:21:32,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ConfusõesDePrimeiraCl...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:20,00:22:20,BR (Brazil)
2,Dimitri,2021-07-11 15:59:26,00:17:28,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:36,00:22:36,BR (Brazil)
3,Dimitri,2021-07-01 22:48:52,00:02:51,NaN,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,Android DefaultWidevineL3Phone Android Phone,00:02:51,Not latest view,BR (Brazil)
4,Dimitri,2021-07-01 20:42:01,01:10:59,NaN,Karate Kid,NaN,Android DefaultWidevineL3Phone Android Phone,01:15:43,01:15:43,BR (Brazil)


In [ ]:
#Averiguando os tipos de dados.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Profile Name             380 non-null    object
 1   Start Time               380 non-null    object
 2   Duration                 380 non-null    object
 3   Attributes               180 non-null    object
 4   Title                    380 non-null    object
 5   Supplemental Video Type  257 non-null    object
 6   Device Type              380 non-null    object
 7   Bookmark                 380 non-null    object
 8   Latest Bookmark          380 non-null    object
 9   Country                  380 non-null    object
dtypes: object(10)
memory usage: 29.8+ KB


In [ ]:
# Convertendo os tipos de dados das colunas start time.
df['Start Time'] = pd.to_datetime(df['Start Time'],utc=True)
df = df.set_index('Start Time')


In [ ]:
#mudando o nome e o tipo de dado para poder contabilizar 
df['duração em minuto'] = df['Duration'].str.split(':').apply(
    lambda x: int(x[0]) * 60.0 + int(x[1]) + int(x[2]) / 60.0)

In [ ]:
# Criando as colunas de duração(hora) e (min)
df['duração em horas'] = df['duração em minuto']/60.0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 380 entries, 2021-07-11 16:39:18+00:00 to 2021-07-01 23:34:26+00:00
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Profile Name             380 non-null    object 
 1   Duration                 380 non-null    object 
 2   Attributes               180 non-null    object 
 3   Title                    380 non-null    object 
 4   Supplemental Video Type  257 non-null    object 
 5   Device Type              380 non-null    object 
 6   Bookmark                 380 non-null    object 
 7   Latest Bookmark          380 non-null    object 
 8   Country                  380 non-null    object 
 9   duração em minuto        380 non-null    float64
 10  duração em horas         380 non-null    float64
dtypes: float64(2), object(9)
memory usage: 35.6+ KB


In [ ]:
df.head()

,Profile Name,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,duração em minuto,duração em horas
Start Time,,,,,,,,,,,
2021-07-11 16:39:18+00:00,Dimitri,00:22:55,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #OsSuperDoidos (Episód...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:15,00:22:15,BR (Brazil),22.916667,0.381944
2021-07-11 16:17:23+00:00,Dimitri,00:21:32,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ConfusõesDePrimeiraCl...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:20,00:22:20,BR (Brazil),21.533333,0.358889
2021-07-11 15:59:26+00:00,Dimitri,00:17:28,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:36,00:22:36,BR (Brazil),17.466667,0.291111
2021-07-01 22:48:52+00:00,Dimitri,00:02:51,NaN,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,Android DefaultWidevineL3Phone Android Phone,00:02:51,Not latest view,BR (Brazil),2.850000,0.047500
2021-07-01 20:42:01+00:00,Dimitri,01:10:59,NaN,Karate Kid,NaN,Android DefaultWidevineL3Phone Android Phone,01:15:43,01:15:43,BR (Brazil),70.983333,1.183056


In [ ]:
# Convertendo para o horário brasileiro
df.index = df.index.tz_convert('America/Sao_Paulo')
df = df.reset_index()


In [ ]:
df.head()

,Start Time,Profile Name,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,duração em minuto,duração em horas
0,2021-07-11 13:39:18-03:00,Dimitri,00:22:55,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #OsSuperDoidos (Episód...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:15,00:22:15,BR (Brazil),22.916667,0.381944
1,2021-07-11 13:17:23-03:00,Dimitri,00:21:32,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ConfusõesDePrimeiraCl...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:20,00:22:20,BR (Brazil),21.533333,0.358889
2,2021-07-11 12:59:26-03:00,Dimitri,00:17:28,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:36,00:22:36,BR (Brazil),17.466667,0.291111
3,2021-07-01 19:48:52-03:00,Dimitri,00:02:51,NaN,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,Android DefaultWidevineL3Phone Android Phone,00:02:51,Not latest view,BR (Brazil),2.850000,0.047500
4,2021-07-01 17:42:01-03:00,Dimitri,01:10:59,NaN,Karate Kid,NaN,Android DefaultWidevineL3Phone Android Phone,01:15:43,01:15:43,BR (Brazil),70.983333,1.183056


In [ ]:
# fragmentando a coluna start time em outras colunas
df['ano'] = df['Start Time'].dt.year
df['mes'] = df['Start Time'].dt.month_name()
df['dia'] = df['Start Time'].dt.day
df['dia da semana'] = df['Start Time'].dt.day_name()
df['hora do dia'] = df['Start Time'].dt.hour


In [ ]:
df.head()

,Start Time,Profile Name,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country,duração em minuto,duração em horas,ano,mes,dia,dia da semana,hora do dia
0,2021-07-11 13:39:18-03:00,Dimitri,00:22:55,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #OsSuperDoidos (Episód...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:15,00:22:15,BR (Brazil),22.916667,0.381944,2021,July,11,Sunday,13
1,2021-07-11 13:17:23-03:00,Dimitri,00:21:32,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ConfusõesDePrimeiraCl...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:20,00:22:20,BR (Brazil),21.533333,0.358889,2021,July,11,Sunday,13
2,2021-07-11 12:59:26-03:00,Dimitri,00:17:28,Autoplayed: user action: User_Interaction;,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,MStar PureTurnkey51 Skyworth OEM2K Smart TV,00:22:36,00:22:36,BR (Brazil),17.466667,0.291111,2021,July,11,Sunday,12
3,2021-07-01 19:48:52-03:00,Dimitri,00:02:51,NaN,Sam & Cat: Temporada 2: #ODroneEOBebê (Episódi...,NaN,Android DefaultWidevineL3Phone Android Phone,00:02:51,Not latest view,BR (Brazil),2.850000,0.047500,2021,July,1,Thursday,19
4,2021-07-01 17:42:01-03:00,Dimitri,01:10:59,NaN,Karate Kid,NaN,Android DefaultWidevineL3Phone Android Phone,01:15:43,01:15:43,BR (Brazil),70.983333,1.183056,2021,July,1,Thursday,17


In [ ]:

# Criando mais duas colunas a partir da coluna título.
df[['Name Title','Temporada/Ep']] = df['Title'].str.split(':',n=1, expand=True)
#df.drop('Title', axis=1, inplace=True)

In [ ]:
# Classificando a coluna type view em filme e serie.
df['Type View'] = df.apply(lambda x:'Filme' if pd.isnull(x['Temporada/Ep']) else 'Serie TV' , axis=1)

In [ ]:
# Eliminando algumas colunas
df = df.drop(columns=['Attributes','Supplemental Video Type','Bookmark','Latest Bookmark'])

In [ ]:
# Renomeando colunas
df = df.rename(columns={'Start Time': 'Data e hora', 'Profile Name':'Nome do Perfil', 'Country':'País','Name Title':'Título','Type View':'Tipo'})

In [ ]:
df = df[(df['Nome do Perfil'] == 'Olavo')]

# **Análise Exploratória**

In [ ]:
df.head(4)

,Data e hora,Nome do Perfil,Duration,Title,Device Type,País,duração em minuto,duração em horas,ano,mes,dia,dia da semana,hora do dia,Título,Temporada/Ep,Tipo
5,2021-07-16 11:58:04-03:00,Olavo,00:00:09,Temporada 3 (Trailer): Explicando,Chrome PC (Cadmium),BR (Brazil),0.150000,0.002500,2021,July,16,Friday,11,Temporada 3 (Trailer),Explicando,Serie TV
6,2021-07-16 11:29:15-03:00,Olavo,00:00:04,Take Your Pills_Autohook_2_16x9,Chrome PC (Cadmium),BR (Brazil),0.066667,0.001111,2021,July,16,Friday,11,Take Your Pills_Autohook_2_16x9,None,Filme
7,2021-07-16 11:28:05-03:00,Olavo,00:00:10,Absorvendo o Tabu (Trailer),Chrome PC (Cadmium),BR (Brazil),0.166667,0.002778,2021,July,16,Friday,11,Absorvendo o Tabu (Trailer),None,Filme
8,2021-07-16 11:27:31-03:00,Olavo,00:00:06,The Cabin in the Woods_hook_primary_16x9,Chrome PC (Cadmium),BR (Brazil),0.100000,0.001667,2021,July,16,Friday,11,The Cabin in the Woods_hook_primary_16x9,None,Filme


In [ ]:
df['Título'].value_counts().head()

Temporada 1 (Trailer)    33
Trailer                  20
The Seven Deadly Sins    18
Temporada 2 (Trailer)    12
Sex/Life                 11
Name: Título, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293 entries, 5 to 297
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype                            
---  ------             --------------  -----                            
 0   Data e hora        293 non-null    datetime64[ns, America/Sao_Paulo]
 1   Nome do Perfil     293 non-null    object                           
 2   Duration           293 non-null    object                           
 3   Title              293 non-null    object                           
 4   Device Type        293 non-null    object                           
 5   País               293 non-null    object                           
 6   duração em minuto  293 non-null    float64                          
 7   duração em horas   293 non-null    float64                          
 8   ano                293 non-null    int64                            
 9   mes                293 non-null    object                           
 10  di

In [ ]:
#Tempo total por aparelho
tempo_aparelho = df.groupby(by=['Título'])['duração em minuto'].sum().sort_values(ascending=False)
tempo_aparelho.head()

Título
Sex/Life                 356.733333
Hollywood                340.116667
The Seven Deadly Sins    251.633333
Lupin                    183.483333
Cidade de Gelo           145.233333
Name: duração em minuto, dtype: float64

In [ ]:
#Tempo total dia da semana
dia_semana = df.groupby(by=["dia da semana"])['duração em minuto'].sum().sort_values(ascending=False)
dia_semana.head()

dia da semana
Monday      370.750000
Thursday    360.900000
Sunday      351.566667
Friday      275.600000
Tuesday     199.350000
Name: duração em minuto, dtype: float64

In [ ]:
#Tempo total no ano
dia_semana = df.groupby(by=["ano"])['duração em minuto'].sum().sort_values(ascending=False)
dia_semana.head()

ano
2021    1734.9
Name: duração em minuto, dtype: float64

In [ ]:
#Tempo total mes
mes = df.groupby(by=["mes"])['duração em minuto'].sum().sort_values(ascending=False)
mes.head()

mes
July    1734.9
Name: duração em minuto, dtype: float64

In [ ]:
#Nº de ep assistidos 
n_ep = df['Temporada/Ep'].value_counts()
n_ep.sum()

214

In [ ]:
filmes = df['Tipo']=='Filme'
filmes.value_counts()

False    214
True      79
Name: Tipo, dtype: int64

In [ ]:
df.head(3)

,Data e hora,Nome do Perfil,Duration,Title,Device Type,País,duração em minuto,duração em horas,ano,mes,dia,dia da semana,hora do dia,Título,Temporada/Ep,Tipo
5,2021-07-16 11:58:04-03:00,Olavo,00:00:09,Temporada 3 (Trailer): Explicando,Chrome PC (Cadmium),BR (Brazil),0.150000,0.002500,2021,July,16,Friday,11,Temporada 3 (Trailer),Explicando,Serie TV
6,2021-07-16 11:29:15-03:00,Olavo,00:00:04,Take Your Pills_Autohook_2_16x9,Chrome PC (Cadmium),BR (Brazil),0.066667,0.001111,2021,July,16,Friday,11,Take Your Pills_Autohook_2_16x9,None,Filme
7,2021-07-16 11:28:05-03:00,Olavo,00:00:10,Absorvendo o Tabu (Trailer),Chrome PC (Cadmium),BR (Brazil),0.166667,0.002778,2021,July,16,Friday,11,Absorvendo o Tabu (Trailer),None,Filme


In [ ]:
df = df.drop(columns='Duration')

In [ ]:
df.to_csv('netflix.csv', index=0)